In [1]:
import csv
import os
from geopy import distance

data_pkg_path = '../data'
input_filename = 'worldcities.csv'
input_path = os.path.join(data_pkg_path, input_filename)

home_city = 'Bengaluru'
home_country = 'India'

import csv
import os
import requests
import time

ORS_API_KEY = '<replace this with your key>'

def get_driving_distance(source_coordinates, dest_coordinates):
    parameters = {
    'api_key': ORS_API_KEY,
    'start' : '{},{}'.format(source_coordinates[1], source_coordinates[0]),
    'end' : '{},{}'.format(dest_coordinates[1], dest_coordinates[0])
    }

    response = requests.get(
        'https://api.openrouteservice.org/v2/directions/driving-car', params=parameters)

    if response.status_code == 200:
        data = response.json()
        summary = data['features'][0]['properties']['summary']
        distance = summary['distance']
        return distance/1000
    else:
        print('Request failed.')
        return -9999
 

def write_distance_file(home_city, home_country, output_filename):
        
    with open(input_path, 'r', encoding='utf-8') as input_file:
        csv_reader = csv.DictReader(input_file)
        for row in csv_reader:
            if row['city_ascii'] == home_city:
                home_city_coordinates = (row['lat'], row['lng'])
                break

    output_filename = '{}_distance.csv'.format(home_city)
    output_dir = '../output'
    output_path = os.path.join(output_dir, output_filename)

    with open(output_path, mode='w') as output_file:
        fieldnames = ['city', 'distance_from_home']
        csv_writer = csv.DictWriter(output_file, fieldnames=fieldnames)
        csv_writer.writeheader()

        with open(input_path, 'r', encoding='utf-8') as input_file:
            csv_reader = csv.DictReader(input_file)
            for row in csv_reader:
                if (row['country'] == home_country and
                    row['city_ascii'] != home_city):
                    city_coordinates = (row['lat'], row['lng'])
                    city_distance = get_driving_distance(
                        city_coordinates, home_city_coordinates)
                    print(row['city_ascii'], city_distance)
                    time.sleep(2)
                    csv_writer.writerow(
                        {'city': row['city_ascii'],
                         'distance_from_home': city_distance}
                    )

    print('Successfully written output file at {}'.format(output_path))


write_distance_file('Colombo', 'Sri Lanka', 'srilanka_distance.csv')

Successfully written output file at ../output/Colombo_distance.csv
